In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("donations_byzip.csv")


In [62]:
df.head(1)

,ZipCode,GivingRatio,45_64,65_Plus,TotalContributions,TotalAGI,MedianContribution,MedianAGI,Population,Under18,18_24,25_44
0,10026,2.22,23.6,8.2,14253000,643001000,2791,80630,36797,21.5,11.6,35.1


In [6]:
"""converts column to float
    removes ($ , %)
    returns column
"""
def float_convert(column):
    return column.replace( '[\$,\%)]','', regex=True ).astype(float)

# divide by max
def scale(column):
    floated = float_convert(column)
    return floated / max(floated)


In [11]:

# drop index and url columns
#df = df.drop(['index'], axis=1)
#df = df.drop(['url'], axis=1)

# convert all remaining columns to float
df = df.apply(float_convert)


In [ ]:
"""
NAICS codes:
54 - Professional, Scientific, Technical Services
    The establishments in this sector specialize according to expertise 
and provide these services to clients
in a variety of industries and, in some cases, to households. Activities
performed include: legal advice and representation; accounting, 
bookkeeping, and payroll services; architectural, engineering, 
and specialized design services; computer services; consulting services;
research services; advertising services; photographic services;
translation and interpretation services; veterinary services; 
and other professional, scientific, and technical services.

51 - Information
    The Information sector comprises establishments engaged in the following
processes: (a) producing and distributing information and cultural
products, (b) providing the means to transmit or distribute these products
as well as data or communications, and (c) processing data.

52 - Finance and Insurance
    The Finance and Insurance sector comprises establishments primarily
engaged in financial transactions (transactions involving the creation,
liquidation, or change in ownership of financial assets) and/or in
facilitating financial transactions.

more information: http://www.naics.com/search/
"""

In [12]:
census = pd.read_csv("zbp13detail.txt")

In [16]:
#totals.loc[totals['zip'] == 11206]
census.head(1)
# zip: zipcode
# naics: industry code
# est: total number of establishments in zip
# nX_Y: number of establishments with between X and Y employees

,zip,naics,est,n1_4,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000
0,501,------,2,2,0,0,0,0,0,0,0,0


In [17]:
totals = census.loc[census['naics'] == '------']
tech = (census.loc[census['naics'] == '54----'] )
info = (census.loc[census['naics'] == '51----'] )
finance = (census.loc[census['naics'] == '52----'] )

In [18]:
# get total number of establishments
# returns float containing number of establishments in specified df at zip
def get_num_est(df, zipcode):
    row = df.loc[df['zip'] == zipcode]
    return float(row['est'])


def get_tech_ratio(zipcode):
    tech_est = get_num_est(tech, zipcode)
    info_est = get_num_est(info, zipcode)
    finance_est = get_num_est(finance, zipcode)
    industry_totals = tech_est + info_est + finance_est
    
    total_est = get_num_est(totals, zipcode)
    return float(industry_totals / total_est)

# 
#def get_ratio(zipcode, idlist, idtotal):
    



In [21]:
# import csv containing station key with addresses
zipdata = pd.read_csv("zipcode_data_clean.csv")

In [22]:
# create tech ratio column
zipdata['tech_ratio'] = zipdata.apply(lambda row: get_tech_ratio(row['ZipCode']), axis=1)

In [64]:
zipdata.head(1)

,Unnamed: 0,ZipCode,GivingRatio,45_64,65_Plus,TotalContributions,TotalAGI,MedianContribution,MedianAGI,Population,Under18,18_24,25_44,tech_ratio
0,0,10026,2.22,23.6,8.2,14253000,643001000,2791,80630,36797,21.5,11.6,35.1,0.099352


In [65]:
bk_man = pd.read_csv("bk_manhattan2.csv")

510

In [87]:
list(bk_man.columns.values)

['Unnamed: 0',
 'Unnamed: 0.1',
 'UNIT',
 'C/A',
 'LOCATION',
 'NUM',
 'LINE',
 'LAT',
 'LONG',
 'geo_tuple',
 'geo_address',
 'ZipCode',
 'GivingRatio',
 '45_64',
 '65_Plus',
 'TotalContributions',
 'TotalAGI',
 'MedianContribution',
 'MedianAGI',
 'Population',
 'Under18',
 '18_24',
 '25_44',
 'index',
 'url',
 'MedianAGI_f',
 'MedianCont_f',
 'AGI_Scaled',
 'Contributions_Scaled',
 'TotalCont_Scaled',
 'TotalAGI_Scaled',
 'MedianCont_Scaled',
 'MedianAGI_Scaled',
 'Population_Scaled']

In [32]:
# only relevant columns
stations_geo = bk_man[['UNIT','C/A','LOCATION','NUM','LINE','LAT','LONG',
                'geo_tuple','geo_address','ZipCode']]


In [37]:
stations_geo.head(1)

,UNIT,C/A,LOCATION,NUM,LINE,LAT,LONG,geo_tuple,geo_address,ZipCode
0,R216,C027,BAY RIDGE-95 ST,R,BMT,40.616624,-74.030964,"(40.616624000000002, -74.030963999999997)","9404, 4th Avenue, Fort Hamilton, Kings County,...",11209


In [35]:
df_mer2 = pd.merge(left=stations_geo, right=zipdata, how='left', left_on='ZipCode', right_on='ZipCode')

In [36]:
df_mer2.head(1)

,UNIT,C/A,LOCATION,NUM,LINE,LAT,LONG,geo_tuple,geo_address,ZipCode,...,65_Plus,TotalContributions,TotalAGI,MedianContribution,MedianAGI,Population,Under18,18_24,25_44,tech_ratio
0,R216,C027,BAY RIDGE-95 ST,R,BMT,40.616624,-74.030964,"(40.616624000000002, -74.030963999999997)","9404, 4th Avenue, Fort Hamilton, Kings County,...",11209,...,14.3,31554000,1553612000,2363,86557,72434,20,7.5,32.5,0.141104


In [66]:
# concat location and num
df_mer2['LOCNUM'] = df_mer2[['LOCATION', 'NUM']].apply(lambda x: ''.join(x), axis=1)


In [ ]:
#df_fin = bk_man[['UNIT','C/A','LOCATION','NUM','LINE','LAT','LONG',
#                'geo_tuple','geo_address','ZipCode','GivingRatio',
#                'TotalContributions','TotalAGI','MedianContribution',
#                'MedianAGI','Population','Under18','18_24','25_44',
#                '45_64','65_Plus']]

In [82]:
entry_totals = pd.read_csv("dan_station_weekly2.csv")
#entry_totals = pd.read_csv("weekly_sum_totals.csv")

In [83]:
entry_totals['LOCNUM'] = entry_totals[['STATION', 'LINENAME']].apply(lambda x: ''.join(x), axis=1)

In [84]:
entry_totals.head(1)
len(entry_totals)

3288

In [71]:
df_mer3 = pd.merge(left=df_mer2, right=entry_totals, how='left', left_on='LOCATION', right_on='STATION')

In [73]:
df_mer3.head(1)

510

In [74]:
import numpy as np

# normalize median contribution, entries, and median income
df_mer3['Med_Cont_Nor'] = float_convert(df_mer3['MedianContribution'][pd.notnull(df_mer3['MedianContribution'])])
df_mer3['ENTRIES_Nor'] = float_convert(df_mer3['ENTRIES'][pd.notnull(df_mer3['ENTRIES'])])
df_mer3['Median_AGI_Nor'] = float_convert(df_mer3['MedianAGI'][pd.notnull(df_mer3['MedianAGI'])])

# median contribution *** % of population age 25-44 
# *** average daily ridership *** tech ratio
df_mer3['VALUE'] = (df_mer3['tech_ratio'] * df_mer3['Med_Cont_Nor'] *
                    (df_mer3['25_44'] / 100) * df_mer3['ENTRIES_Nor'])

In [75]:
# top 25, by tech_ratio
fdf = df_mer3

In [76]:
len(fdf)

510

In [77]:
fdf.loc[fdf['UNIT'] == 'R334']

,UNIT,C/A,LOCATION,NUM,LINE,LAT,LONG,geo_tuple,geo_address,ZipCode,...,25_44,tech_ratio,LOCNUM,STATION,Unnamed: 0_y,ENTRIES,Med_Cont_Nor,ENTRIES_Nor,Median_AGI_Nor,VALUE
396,R334,N035,CATHEDRL-110 ST,BC,IND,40.800637,-73.958201,"(40.800637000000002, -73.958201000000003)","Frederick Douglass Circle, Park West Village, ...",10026,...,35.1,0.099352,CATHEDRL-110 STBC,NaN,NaN,NaN,2791,NaN,80630,NaN
397,R334,N034,CATHEDRL-110 ST,BC,IND,40.800637,-73.958201,"(40.800637000000002, -73.958201000000003)","Frederick Douglass Circle, Park West Village, ...",10026,...,35.1,0.099352,CATHEDRL-110 STBC,NaN,NaN,NaN,2791,NaN,80630,NaN
